In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc 
from sklearn.model_selection import train_test_split
from pandas.plotting import register_matplotlib_converters




# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
df = pd.read_csv("../input/trafficp/Finalset1csvpred.csv",parse_dates =['5 Minutes'],index_col ='5 Minutes' )

In [3]:
df.head()

In [4]:
df.drop('Lane 1 Flow (Veh/Minute)',axis='columns', inplace=True)
df.drop('Unnamed: 3',axis='columns', inplace=True)
df.drop('Unnamed: 4',axis='columns', inplace=True)
df.drop('Unnamed: 5',axis='columns', inplace=True)
df.drop('Unnamed: 6',axis='columns', inplace=True)
df.drop('Unnamed: 7',axis='columns', inplace=True)
df.drop('Unnamed: 8',axis='columns', inplace=True)
df.drop('Unnamed: 9',axis='columns', inplace=True)
df.drop('Unnamed: 10',axis='columns', inplace=True)
df.drop('Unnamed: 11',axis='columns', inplace=True)
df.drop('Unnamed: 12',axis='columns', inplace=True)
df.drop('Unnamed: 13',axis='columns', inplace=True)
df.drop('Unnamed: 14',axis='columns', inplace=True)
df.drop('Unnamed: 15',axis='columns', inplace=True)
df.drop('Unnamed: 16',axis='columns', inplace=True)
df.drop('Unnamed: 17',axis='columns', inplace=True)
df.drop('Unnamed: 18',axis='columns', inplace=True)
df.drop('Unnamed: 19',axis='columns', inplace=True)
df.drop('Unnamed: 20',axis='columns', inplace=True)
df.drop('Unnamed: 21',axis='columns', inplace=True)


In [5]:
df

In [6]:
df.index

In [7]:
df = pd.DataFrame(df)

print(df)

# drop all rows with any NaN and NaT values
df1 = df.dropna()
df = df1
print(df)

In [8]:
#Adding one more column for day of week using timestamps

df["day_of_week"] = df.index.dayofweek

In [9]:
df

In [10]:
#plotting data for speed vs time

sns.lineplot(x=df.index, y="Lane 1 Speed (mph)",data=df)

In [11]:
#scatter plot

sns.pointplot(data=df, x=df.index, y="Lane 1 Speed (mph)")

In [12]:
#Plot for avg speed vs day of week

sns.pointplot(data=df, x="day_of_week", y="Lane 1 Speed (mph)")

In [13]:
#Splitting the data for training

train_size = int(len(df) * 0.75) #0.75  shows 75% data is to be used for training
test_size = len(df) - train_size

train,test = df.iloc[0:train_size], df.iloc[train_size:len(df)]

print(train.shape, test.shape)

In [14]:
#Data preprocessing

from sklearn.preprocessing import RobustScaler

#f_columns = ['Lane 1 Flow (Veh/Minute)']

#f_transformer = RobustScaler()
#cnt_transformer = RobustScaler()

#f_transformer = f_transformer.fit(train[f_columns].to_numpy())
#cnt_transformer = cnt_transformer.fit(train[['Lane 1 Speed (mph)']])

#train.loc[:, f_columns] = f_transformer.transform(train[f_columns].to_numpy())
#train['Lane 1 Speed (mph)'] = cnt_transformer.transform(train['Lane 1 Speed (mph)'])

#f_transformer = f_transformer.fit(train[f_columns].to_numpy())
#cnt_transformer = cnt_transformer.fit(train[['Lane 1 Speed (mph)']])

#test.loc[:, f_columns] = f_transformer.transform(test[f_columns].to_numpy())
#test['Lane 1 Speed (mph)'] = cnt_transformer.transform(test['Lane 1 Speed (mph)'])


In [15]:
cnt_transformer = RobustScaler()
cnt_transformer = cnt_transformer.fit(train[['Lane 1 Speed (mph)']])
train['Lane 1 Speed (mph)'] = cnt_transformer.transform(train[['Lane 1 Speed (mph)']])
test['Lane 1 Speed (mph)'] = cnt_transformer.transform(test[['Lane 1 Speed (mph)']])

In [16]:
def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)
        ys.append(y.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)

In [17]:
time_steps = 1

# reshape to [samples, time_steps, n_features]

X_train, y_train = create_dataset(train, train['Lane 1 Speed (mph)'], time_steps)
X_test, y_test = create_dataset(test, test['Lane 1 Speed (mph)'], time_steps)
print(X_train.shape, y_train.shape)

In [18]:
print(X_test.shape, y_test.shape)

In [19]:
#Building LSTM model

model = keras.Sequential()
model.add(
  keras.layers.Bidirectional(
    keras.layers.LSTM(
      units=100,                                                     #number of neurons
      input_shape=(X_train.shape[1], X_train.shape[2])
    )
  )
)
model.add(keras.layers.Dropout(rate=0.2))
model.add(keras.layers.Dense(units=1))
model.compile(loss='mean_squared_error', optimizer='adam')

In [20]:
history = model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=60,
    validation_split=0.1,
    shuffle=False
)

In [21]:
plt.plot(history.history['loss'],label = 'train')
plt.plot(history.history['loss'],label = 'validation')
plt.legend()

In [22]:
y_pred = model.predict(X_test)

In [23]:
y_train_inv = cnt_transformer.inverse_transform(y_train.reshape(1, -1))
y_test_inv = cnt_transformer.inverse_transform(y_test.reshape(1, -1))
y_pred_inv = cnt_transformer.inverse_transform(y_pred)

In [24]:
import pylab
pylab.plot(np.arange(0, len(y_train)), y_train_inv.flatten(), 'g', label="history")
pylab.plot(np.arange(len(y_train), len(y_train) + len(y_test)), y_test_inv.flatten(), marker='.', label="true")
pylab.plot(np.arange(len(y_train), len(y_train) + len(y_test)), y_pred_inv.flatten(), 'r', label="prediction")
pylab.ylabel('Vehicle Speed(mph)')
pylab.xlabel('Time Step')
pylab.legend()
pylab.ylim(0,100)
pylab.show();

In [25]:
import pylab
pylab.plot(y_test_inv.flatten(), marker='.', label="true")
pylab.plot(y_pred_inv.flatten(), 'r', label="prediction")
pylab.ylabel('Vehicle Speed(mph)')
pylab.xlabel('Time Step')
pylab.legend()


pylab.show();

In [26]:
print(y_pred_inv[0][0])

prdlst = []

for i in y_pred_inv:
    prdlst.append(i[0])
    
print(prdlst[0])



In [27]:
truelst = [18]

for x in y_test_inv[0]:
    truelst.append(x)
    
truelst.pop()




In [28]:
newlst = list(zip(truelst,prdlst))

#MAPE Calculation
pelst = []
b=0
for i,j in newlst:
    b = (abs( (i-j) ) /i)*100
    pelst.append(b)
        
maperror = sum(pelst)/len(pelst)
print(maperror)
print(pelst[0])

    

In [29]:
from sklearn.metrics import mean_squared_error
rmse = (mean_squared_error(truelst,prdlst))**0.5
rmse